In [63]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np

In [70]:
batch_size = 32
img_height = 100
img_width = 100

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
    color_mode="grayscale",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/eye_imgs/',
  validation_split=0.2,
  color_mode="grayscale",
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 20151 files belonging to 5 classes.
Using 16121 files for training.
Found 20151 files belonging to 5 classes.
Using 4030 files for validation.


In [71]:
class_names = train_ds.class_names
print(class_names)

['center', 'down', 'left', 'right', 'up']


In [126]:
num_classes = 5

input_layer = keras.Input(shape=(img_height, img_width, 1))
preprocessing = keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
conv1 = keras.layers.Conv2D(16, 1, padding='same', activation='relu')(preprocessing)
pool1 = keras.layers.MaxPooling2D()(conv1)
conv2 = keras.layers.Conv2D(32, 1, padding='same', activation='relu')(pool1)
pool2 = keras.layers.MaxPooling2D()(conv2)
conv3 = layers.Conv2D(64, 1, padding='same', activation='relu')(pool2)
pool3 = keras.layers.MaxPooling2D()(conv3)
dropout = keras.layers.Dropout(0.2)(pool3)
flatten = keras.layers.Flatten()(dropout)
dense = keras.layers.Dense(128, activation='relu')(flatten)
output_layer = keras.layers.Dense(num_classes)(dense)

model = keras.Model(inputs=input_layer, outputs=output_layer, name="jupiter2")

opt = tf.keras.optimizers.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    
model.compile(optimizer="adam", 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

for i in range(0,10):
    model.layers[i].trainable = False
    
model.compile(optimizer="adam", 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "jupiter2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 100, 100, 1)]     0         
_________________________________________________________________
rescaling_13 (Rescaling)     (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 100, 100, 16)      32        
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 50, 50, 32)        544       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 25, 25, 64)        211

In [127]:
from tensorflow.keras.applications import ResNet50
num_classes = 5

# Initialize the Pretrained Model
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(img_height, img_width, 1),
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer
input_ = tf.keras.Input(shape=(img_height, img_width, 1))

# Set the feature extractor layer
x = feature_extractor(input_, training=False)

# Set the pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(num_classes)(x)

# Create the new model object
model = tf.keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

ValueError: The input must have 3 channels; got `input_shape=(100, 100, 1)`

In [76]:
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
504/504 [==============================] - 46s 90ms/step - loss: 0.0891 - accuracy: 0.9664 - val_loss: 0.1895 - val_accuracy: 0.9464
Epoch 2/10
504/504 [==============================] - 44s 88ms/step - loss: 0.0906 - accuracy: 0.9676 - val_loss: 0.1948 - val_accuracy: 0.9429
Epoch 3/10
504/504 [==============================] - 46s 91ms/step - loss: 0.0866 - accuracy: 0.9686 - val_loss: 0.2142 - val_accuracy: 0.9447
Epoch 4/10
504/504 [==============================] - 45s 88ms/step - loss: 0.0830 - accuracy: 0.9708 - val_loss: 0.1867 - val_accuracy: 0.9479
Epoch 5/10
504/504 [==============================] - 44s 87ms/step - loss: 0.0796 - accuracy: 0.9713 - val_loss: 0.2178 - val_accuracy: 0.9449
Epoch 6/10
504/504 [==============================] - 39s 77ms/step - loss: 0.0805 - accuracy: 0.9706 - val_loss: 0.1923 - val_accuracy: 0.9519
Epoch 7/10
504/504 [==============================] - 38s 75ms/step - loss: 0.0757 - accuracy: 0.9718 - val_loss: 0.1992 - val_accuracy:

In [ ]:
model.

In [79]:
len(model.layers)

12

In [19]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
504/504 [==============================] - 37s 74ms/step - loss: 1.0064 - accuracy: 0.9001 - val_loss: 1.0257 - val_accuracy: 0.8774
Epoch 2/5
504/504 [==============================] - 37s 74ms/step - loss: 1.0009 - accuracy: 0.9045 - val_loss: 1.0054 - val_accuracy: 0.8998
Epoch 3/5
504/504 [==============================] - 36s 72ms/step - loss: 0.9967 - accuracy: 0.9096 - val_loss: 0.9975 - val_accuracy: 0.9087
Epoch 4/5
504/504 [==============================] - 38s 75ms/step - loss: 0.9915 - accuracy: 0.9147 - val_loss: 0.9875 - val_accuracy: 0.9179
Epoch 5/5
504/504 [==============================] - 44s 88ms/step - loss: 0.9883 - accuracy: 0.9179 - val_loss: 0.9858 - val_accuracy: 0.9191


In [24]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
504/504 [==============================] - 48s 95ms/step - loss: 0.9736 - accuracy: 0.9315 - val_loss: 0.9773 - val_accuracy: 0.9285
Epoch 2/5
504/504 [==============================] - 47s 94ms/step - loss: 0.9688 - accuracy: 0.9361 - val_loss: 0.9785 - val_accuracy: 0.9266
Epoch 3/5
504/504 [==============================] - 39s 77ms/step - loss: 0.9762 - accuracy: 0.9289 - val_loss: 0.9808 - val_accuracy: 0.9236
Epoch 4/5
504/504 [==============================] - 38s 75ms/step - loss: 0.9723 - accuracy: 0.9328 - val_loss: 0.9760 - val_accuracy: 0.9283
Epoch 5/5
504/504 [==============================] - 39s 77ms/step - loss: 0.9699 - accuracy: 0.9347 - val_loss: 0.9781 - val_accuracy: 0.9268


In [97]:
model.save('./Models/jupiter2')

INFO:tensorflow:Assets written to: ./Models/jupiter2/assets


In [98]:
newmodel = tf.keras.models.load_model('./Models/jupiter2/')

In [104]:
def load_model_for_retrain():
    model = tf.keras.models.load_model('./Models/jupiter2/')
    # make all but last two layers untrainable
    for i in range(0,10):
        model.layers[i].trainable = False
    model.compile(optimizer="adam", 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [ ]:
def retrain():
    
    batch_size = 32
    img_height = 100
    img_width = 100

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      '../temp/data',
      validation_split=0.1,
        color_mode="grayscale",
      subset="training",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      '../temp/data',
      validation_split=0.1,
      color_mode="grayscale",
      subset="validation",
      seed=123,
      image_size=(img_height, img_width),
      batch_size=batch_size)
    
    model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )
    
    return model

In [112]:
img1 = cv2.imread('./a22.jpg',cv2.IMREAD_UNCHANGED)
# img2 = cv2.imread('../data/eye_imgs/left/a10.jpg' ,cv2.IMREAD_UNCHANGED)
# img = img.reshape((100,100,1))
# img = np.expand_dims(img,0)

In [114]:
img1 = np.expand_dims(img1,0)

In [125]:
np.argmax(model.predict(img1))

0

In [119]:
arr = np.array([1,2,3,4,5,1])

In [124]:
from collections import Counter
Counter(arr).most_common(1)[0][0]

1

In [24]:
def joint_prediction():
    pass

In [42]:

img2 = np.expand_dims(img2,0)

In [43]:
a = np.concatenate((img1,img2))

In [27]:
a.shape

(2, 100, 100)

In [45]:
model.predict(a)

array([ 2.0141094, -0.907617 ,  3.9984462, -2.9328032,  1.9947188],
      dtype=float32)

In [53]:
def predict_both(left, right):
    img1 = np.expand_dims(left,0)
    img2 = np.expand_dims(right,0)
    concat = np.concatenate((img1,img2))
    classes = model.predict_classes(concat)
    if classes[0] == classes[1]:
        return classes[0]
    else:
        pred = model.predict(concat)
        a = (pred[0].argmax(), pred[0].max())
        b = (pred[1].argmax(), pred[1].max())
        if a[1] > b[1]:
            return a[0]
        else:
            return b[0]

In [ ]:
def predict_frame(self, eye_img):
    img = eye_img.copy()
    batch = np.expand_dims(img,0)
    prediction = self.model.predict_classes(batch)[0]
    return prediction

In [211]:
class_names

['center', 'down', 'left', 'right', 'up']

In [212]:
model.predict_classes(img)[0]

2